# LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재사용할 수 있도록 해주는 도구
    단일 입력: 하나의 변수로 구성된 프롬프트 템플릿
    다중 입력: 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트: 시스템/사용자 역할 별 프롬프트 구성 .from_message()
    PartialPromptTemplate 활용: 프롬프트 일부를 미리 고정하고 부분 포멧팅을 사용( ex. 시스템 메시지는 고정.. )
    프롬프트를 출력 및 체인 실행: LCEL
    프롬프트 작성 팁: 주의사항 및 모범사례
```

In [1]:
# 환경 설정
%pip install langchain langchain-openai python-dotenv -qqq

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 단일 프롬프트

In [3]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터입니다.\n"
    "아래 제품의 매력적인 홍보 문구를 100자 이내로 작성해주세요.\n\n"
    "제품 명: {product_name}\n"
)

# 템플릿 객체 생성
product_prompt = PromptTemplate.from_template(template_str)

# 프롬프트에 제품 이름을 삽입
product_name = "스마트폰"
formatted_prompt = product_prompt.format(product_name=product_name)
# 프롬프트 출력
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다.
아래 제품의 매력적인 홍보 문구를 100자 이내로 작성해주세요.

제품 명: 스마트폰



In [ ]:
# 프롬프트 | llm  -> invoke()
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
# OpenAI API 클라이언트 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 출력 파서 설정
parser = StrOutputParser()

# LLM에 프롬프트 전달 및 응답 생성
chain = product_prompt | llm | parser
response = chain.invoke({"product_name": product_name})

# 응답 출력
print(response)

"세상을 손안에! 혁신적인 스마트폰으로 연결되고, 창조하고, 즐기세요. 당신의 일상이 더욱 특별해집니다!"


## 다중 프롬프트

In [ ]:
# 다중 입력
# 다중 입력 템플릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.\n\n"
    "제목: {title}\n"
    "키워드: {keywords}\n"
)

# 다중 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keywords"])
# summary_prompt = PromptTemplate.from_template(multi_template_str)

# 포멧팅을 통해 프롬프트 값 설정
title = "AI 기술의 발전과 미래"
keywords = "인공지능, 머신러닝, 딥러닝"
formatted_summary_prompt = summary_prompt.format(title=title, keywords=keywords)

# 프롬프트 출력
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.

제목: AI 기술의 발전과 미래
키워드: 인공지능, 머신러닝, 딥러닝



In [12]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    "title": title,
    "keywords": keywords
})

print(result_summary)

AI 기술의 발전은 인공지능, 머신러닝, 딥러닝의 혁신적인 발전을 통해 이루어지고 있으며, 이는 다양한 산업 분야에서 큰 변화를 가져오고 있습니다. 특히, 머신러닝과 딥러닝 기술의 진화는 데이터 분석의 정확성과 효율성을 높여주어, 기업들이 보다 스마트한 의사결정을 할 수 있도록 지원하고 있습니다. 앞으로 이러한 기술들은 더욱 발전하여 인간의 삶을 개선하고, 새로운 기회를 창출하는 데 기여할 것으로 기대됩니다.


# ChatPromptTemplate와 역할 기반 프롬프트
```
시스템/사용자/어시스턴트 역할( role )
시스템 메시지: 모델의 동작을 지시
사용자 메시지: 실제 사용자의 입력
어시스턴트 메시지: 이전 모델이 응답한 내용이 있다면 대화 맥락 유지를 위해 활용
```

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

system_message = (
    "당신은 Python 분야의 뛰어난 전문자이자 조언가입니다.\n"
    "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해주세요.\n"
)
user_message = ("{question}")
# 채팅 프롬프트 템플릿 생성
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_message)
])

# OpenAI API 클라이언트 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 출력 파서 설정
parser = StrOutputParser()

# 템플릿을 이용해서 문장을 완성
question = "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?"

# 프롬프트 | llm | parser
chain = chat_prompt | llm | parser

response = chain.invoke({"question": question})
# 응답 출력
print(response)

파이썬에서 리스트를 정렬하는 방법은 여러 가지가 있습니다. 가장 일반적으로 사용되는 두 가지 방법은 `sort()` 메서드와 `sorted()` 함수를 사용하는 것입니다.

### 1. `sort()` 메서드
`sort()` 메서드는 리스트 객체의 메서드로, 리스트를 직접 정렬합니다. 이 메서드는 리스트를 오름차순으로 정렬하며, 원본 리스트를 변경합니다.

```python
# 예시
my_list = [5, 2, 9, 1, 5, 6]
my_list.sort()  # 리스트를 정렬
print(my_list)  # 출력: [1, 2, 5, 5, 6, 9]
```

- **내림차순 정렬**: `reverse=True` 인자를 추가하면 내림차순으로 정렬할 수 있습니다.

```python
my_list.sort(reverse=True)
print(my_list)  # 출력: [9, 6, 5, 5, 2, 1]
```

### 2. `sorted()` 함수
`sorted()` 함수는 리스트를 정렬하여 새로운 리스트를 반환합니다. 원본 리스트는 변경되지 않습니다.

```python
# 예시
my_list = [5, 2, 9, 1, 5, 6]
sorted_list = sorted(my_list)  # 정렬된 새로운 리스트 생성
print(sorted_list)  # 출력: [1, 2, 5, 5, 6, 9]
print(my_list)      # 원본 리스트는 변경되지 않음: [5, 2, 9, 1, 5, 6]
```

- **내림차순 정렬**: `reverse=True` 인자를 추가하여 내림차순으로 정렬할 수 있습니다.

```python
sorted_list_desc = sorted(my_list, reverse=True)
print(sorted_list_desc)  # 출력: [9, 6, 5, 5, 2, 1]
```

### 추가 옵션
두 방법 모두 `key` 인자를 사용하여 정렬 기준을 지정할 수 있습니다. 예를 들어, 문자열 리스트를 길이에 따라 정렬할 수 있습니다.

# PartialPromptTemplate
- 템플릿의 일부를 부분적으로 채운 새로운 템플릿

In [20]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
role_system_template = "당신은 {rule} 분야의 전문 지식인 입니다. 가능한 자세히 답변해주세요."
system_prompt =SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

# ChatPromptTemplate 생성
base_chat_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt
])

partial_chat_prompt = base_chat_prompt.partial(rule="주식 투자")

In [24]:
# partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = "현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떤가요?"
message = partial_chat_prompt.format_messages(question=sample_question)
# 메시지 출력
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떤가요?', additional_kwargs={}, response_metadata={})]


In [25]:
# LCEL 
chain = partial_chat_prompt | llm | parser
response = chain.invoke({"question": sample_question})
# 응답 출력
print(response)

2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

1. **반도체 시장 동향**: 삼성전자는 세계 최대의 반도체 제조업체 중 하나입니다. 반도체 수요는 AI, 클라우드 컴퓨팅, IoT(사물인터넷) 등 다양한 산업에서 증가하고 있습니다. 2025년까지 이러한 기술들이 더욱 발전하고 대중화된다면, 삼성전자의 반도체 부문 실적이 긍정적일 가능성이 높습니다.

2. **스마트폰 및 전자제품 시장**: 삼성전자는 스마트폰, 가전제품 등 다양한 전자제품을 제조하고 있습니다. 5G와 같은 새로운 기술의 도입이 소비자 수요에 긍정적인 영향을 미칠 수 있으며, 특히 프리미엄 제품군에서의 경쟁력이 중요합니다.

3. **글로벌 경제 상황**: 글로벌 경제의 성장률, 금리, 인플레이션 등은 삼성전자의 주가에 영향을 미칠 수 있습니다. 경제가 안정적이고 성장세를 보인다면 소비자 지출이 증가하고, 이는 삼성전자의 매출 증가로 이어질 수 있습니다.

4. **경쟁 상황**: 애플, 화웨이, TSMC 등과 같은 경쟁사들과의 경쟁 상황도 중요합니다. 특히 반도체 분야에서는 TSMC와의 경쟁이 치열해지고 있으며, 기술력과 생산능력에서의 우위가 주가에 큰 영향을 미칠 수 있습니다.

5. **정치적 및 규제 환경**: 한국과 미국, 중국 간의 무역 관계 및 규제 환경도 삼성전자의 사업에 영향을 미칠 수 있습니다. 특히 반도체와 관련된 기술 규제가 강화될 경우, 삼성전자의 글로벌 경쟁력에 부정적인 영향을 미칠 수 있습니다.

6. **ESG(환경, 사회, 지배구조) 경영**: 최근 투자자들은 ESG 요소를 중요하게 고려하고 있습니다. 삼성전자가 지속 가능한 경영을 통해 긍정적인 이미지를 구축한다면, 장기적으로 주가에 긍정적인 영향을 미칠 수 있습니다.

결론적으로, 삼성전자의 주식 전망은 긍정적일 수 있지만, 위에서 언급한 다양한 외부 요인에 따라 변동성이 클 수 있습니다. 따라